# Data Transformation

Chama has some event data come as JSONs, and it needs some transformations to be structured as tables. Convert the presented case.json file to three CSV files, with the following rules:

**CuratedOfferOptions.csv:**

| Variable Name | Transformation |
|------------|----------|
| CurationProvider | in quotes |
| OfferId | in quotes |
| DealerId | in quotes |
| UniqueOptionId | in quotes |
| OptionId | in quotes |
| IsMobileDealer | without quotes |
| IsOpen | without quotes |
| Eta | in quotes |
| ChamaScore | without quotes |
| ProductBrand | in quotes |
| IsWinner | in quotes |
| MinimumPrice | without quotes |
| MaximumPrice | without quotes |
| DynamicPrice | without quotes |
| FinalPrice | without quotes |
| DefeatPrimaryReason | in quotes |
| DefeatReasons | in quotes |
| EnqueuedTimeSP | DD/MM/YYYY (converted to Brasilian timezone - UTC-3) |



**DynamicPriceOption.csv:**

| Variable Name | Transformation |
|------------|----------|
| Provider | in quotes |
| OfferId | in quotes |
| UniqueOptionId | in quotes |
| BestPrice | without quotes |
| EnqueuedTimeSP | DD/MM/YYYY (converted to Brasilian timezone - UTC-3) |


**DynamicPriceRange.csv:**

| Variable Name | Transformation |
|------------|----------|
| Provider | in quotes |
| OfferId | in quotes |
| MinGlobal | without quotes |
| MinRecommended | without quotes |
| MaxRecommended | without quotes |
| DifferenceMinRecommendMinTheory | without quotes |
| EnqueuedTimeSP | DD/MM/YYYY (converted to Brasilian timezone - UTC-3) |



In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re
import glob
import os.path
import calendar
import matplotlib.pyplot as plt
%matplotlib inline
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.cluster import DBSCAN
from kmodes.kmodes import KModes
from sklearn.preprocessing import MinMaxScaler
from pandas.api.types import is_numeric_dtype
from pandas_profiling import ProfileReport
import dtale
from scipy import stats
import sweetviz as sv
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.act_node_interactivity = 'all'
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_json('case.json')

In [3]:
df.shape

(37, 3)

In [4]:
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [5]:
# transform to Brazilian timezone - for all the csv's we need to create
df["EnqueuedTimeUtc"] = pd.to_datetime(arg=df["EnqueuedTimeUtc"]).dt.tz_convert(tz="Brazil/East")

In [6]:
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [7]:
df["EventName"].value_counts()

DynamicPrice_Result    32
CurateOffer_Result      5
Name: EventName, dtype: int64

We need to create 3 csv's. Two of them will be from DynamicPrice_Result and one from CurateOffer_Result under EventName variable.

### Create CuratedOfferOptions.csv file

In [8]:
CuratedOfferOptions = df[df["EventName"] == "CurateOffer_Result"]
CuratedOfferOptions

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [9]:
CuratedOfferOptions.Payload[3]

'[{"curationProvider":"ByPrice","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea","dealerId":"6517","options":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","optionId":"6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2","isMobileDealer":true,"isOpen":true,"eta":"1:00","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":true,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90},{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","optionId":"6517 || 6517","isMobileDealer":false,"isOpen":false,"eta":"0:01","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":false,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90,"defeatPrimaryReason":"Closed","defeatReasons":["Closed","HasDriverInOffer"]}]},{"curationProvider":"ByPrice","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea","dealerId":"9047","options":[{"uniqueOptionId":"8d0f9262-f543-d0c8-a869-33985ae3ecda","optionId":"9047 || 9047 || ULTRAGAZ","isMobileDealer":false,

**One value from above output**

{"curationProvider":"ByPrice","offerId":"149f0e53-ff85-425f-a01a-8710f06704ea","dealerId":"6517","options":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","optionId":"6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2","isMobileDealer":true,"isOpen":true,"eta":"1:00","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":true,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90},{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","optionId":"6517 || 6517","isMobileDealer":false,"isOpen":false,"eta":"0:01","chamaScore":8.0,"productBrand":"ULTRAGAZ","isWinner":false,"minimumPrice":90.00,"maximumPrice":180.00,"dynamicPrice":91.90,"finalPrice":91.90,"defeatPrimaryReason":"Closed","defeatReasons":["Closed","HasDriverInOffer"]}

In [10]:
def curated_offer(row):
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    for el in payload_row:
        curation_provider = f"\"{el['curationProvider']}\""
        offer_id = f"\"{el['offerId']}\""
        dealer_id = f"\"{el['dealerId']}\""
        for option in el["options"]:
            offer_options = {
                "CurationProvider": curation_provider,
                "OfferId": offer_id,
                "DealerId": dealer_id,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                "OptionId": f"\"{option['optionId']}\"",
                "IsMobileDealer": option["isMobileDealer"],
                "IsOpen": option["isOpen"],
                "Eta": f"\"{option['eta']}\"",
                "ChamaScore": option["chamaScore"],
                "ProductBrand": f"\"{option['productBrand']}\"",
                "IsWinner": option["isWinner"],
                "MinimumPrice": option["minimumPrice"],
                "MaximumPrice": option["maximumPrice"],
                "DynamicPrice": option["dynamicPrice"],
                "FinalPrice": option["finalPrice"]
            }
            if "defeatPrimaryReason" in option:
                offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
                offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""
            else:
                offer_options["DefeatPrimaryReason"] = "\"\""
                offer_options["DefeatReasons"] = "\"\""
            offer_options["EnqueuedTimeSP"] = enqueued_time
            result.append(offer_options)
    return result


flatten = [x for xs in CuratedOfferOptions.apply(curated_offer, axis=1) for x in xs]
CuratedOfferOptions_Final = pd.DataFrame(flatten)

In [11]:
CuratedOfferOptions_Final.head()

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/21
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""Closed""","""['Closed', 'HasDriverInOffer']""",25/08/21
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.00,198.00,99.95,99.95,"""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",25/08/21
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.00,184.00,93.90,93.90,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21


In [12]:
CuratedOfferOptions_Final.shape

(40, 18)

In [13]:
# CuratedOfferOptions_Final.to_csv('CuratedOfferOptions.csv', index=False)

### Create DynamicPriceOption.csv file

In [14]:
DynamicPrice_Option_Range = df[df["EventName"] == "DynamicPrice_Result"]
DynamicPrice_Option_Range.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [15]:
DynamicPrice_Option_Range.Payload[1]

'{"provider":"ApplyDynamicPricePerOption","offerId":"56e0702c-0218-4626-8d3d-ae9d54b4503b","algorithmOutput":[{"uniqueOptionId":"b0e296a9-0590-f0e0-8211-243a2ededb12","bestPrice":92.45},{"uniqueOptionId":"d6562c24-0b37-5fb4-8275-65b7b8b47b87","bestPrice":92.45},{"uniqueOptionId":"8d0f9262-f543-d0c8-a869-33985ae3ecda","bestPrice":92.45},{"uniqueOptionId":"151e59ac-761a-96f5-d2b9-882037a9fd28","bestPrice":94.6},{"uniqueOptionId":"3cd346f4-d297-7568-2e50-d43a8e2fd0a9","bestPrice":94.6},{"uniqueOptionId":"b7a7b6d1-4dae-7392-5aaf-f3369c29db1d","bestPrice":93.0},{"uniqueOptionId":"577e4bbd-f49d-ac23-56a6-e70072a05229","bestPrice":93.0},{"uniqueOptionId":"f9b876ab-2590-952f-d69d-5b352ec251f3","bestPrice":91.35}]}'

In [16]:
def dynamic_price_option(row):
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result


lst = DynamicPrice_Option_Range.apply(dynamic_price_option, axis=1)
flatten = [x for xs in lst for x in xs]
DynamicPriceOption_Final = pd.DataFrame(flatten)

In [17]:
DynamicPriceOption_Final.head()

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21


In [18]:
DynamicPriceOption_Final.shape

(56, 5)

In [19]:
# DynamicPriceOption_Final.to_csv('DynamicPriceOption.csv', index=False)

### Create DynamicPriceRange.csv file

In [20]:
def dynamic_price_range(row):
    """Transform the JSON payload into a dynamic price range dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPriceRange":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        algorithm_output = payload_row["algorithmOutput"]
        price_range = {
            "Provider": provider,
            "OfferId": offer_id,
            "MinGlobal": algorithm_output["min_global"],
            "MinRecommended": algorithm_output["min_recommended"],
            "MaxRecommended": algorithm_output["max_recommended"],
            "DifferenceMinRecommendMinTheory": algorithm_output["differenceMinRecommendMinTheory"],
            "EnqueuedTimeSP": enqueued_time
        }
        result.append(price_range)
    return result


lst = DynamicPrice_Option_Range.apply(dynamic_price_range, axis=1)
flatten = [x for xs in lst for x in xs]
DynamicPriceRange_Final = pd.DataFrame(flatten)

In [21]:
DynamicPriceRange_Final.head()

,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""",85.00,87.20,97.65,2.2,05/09/21
1,"""ApplyDynamicPriceRange""","""b8c636fa-8241-47dc-ac40-bdf438a04d9c""",85.00,87.20,97.65,2.2,05/09/21
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""",85.00,87.20,97.65,2.2,05/09/21
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""",85.00,87.20,97.65,2.2,05/09/21
4,"""ApplyDynamicPriceRange""","""fdcfde5c-113d-4a59-9ae0-8bc31e2943d8""",87.35,89.25,99.95,1.9,05/09/21


In [22]:
DynamicPriceRange_Final.shape

(25, 7)

In [23]:
# DynamicPriceRange_Final.to_csv('DynamicPriceRange.csv', index=False)